# Cotações

#### Desafio: 

Faremos um Web Scraping para pegar toda as cotações (Dola, Euro e Ouro) e atualizar os valores com suas respectivas cotações na base de dados.

## Importando Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep

## Configurando padrões do navegador

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)

## Pesquisas

### Abrindo o navegador

In [3]:
browser.get("https://www.google.com/")

### Cotação do dolar

In [4]:
browser.find_element(By.XPATH,
                      '//*[@id="APjFqb"]').send_keys('Cotação dolar' + Keys.ENTER)
cotacaoDolar = browser.find_element(By.XPATH, 
                                    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

### Cotação Euro

In [5]:
browser.find_element(By.CSS_SELECTOR,
                      '#knowledge-currency__updatable-data-column > div.ePzRBb > div > div.vLqKYe.egcvbb.q0WxUd > div > select').click()
sleep(0.2)
browser.find_element(By.CSS_SELECTOR,
                      '#knowledge-currency__updatable-data-column > div.ePzRBb > div > div.vLqKYe.egcvbb.q0WxUd > div > select').send_keys("eur" + Keys.ENTER)
sleep(0.2)
cotacaoEuro = browser.find_element(By.XPATH,
                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

### Cotação Ouro

In [6]:
browser.get('https://www.melhorcambio.com/ouro-hoje')
cotacaoOuro = browser.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute("value")
cotacaoOuro = cotacaoOuro.replace(',', '.')
print(cotacaoOuro)

304.39


### Exibindo Dados

In [7]:
print(f"Cotação Ouro: R${cotacaoOuro}")
sleep(0.5)
print(f'Cotação Euro: R${cotacaoEuro}')
sleep(0.5)
print(f'Cotação dolar: R${cotacaoDolar}')

Cotação Ouro: R$304.39
Cotação Euro: R$5.26129488
Cotação dolar: R$4.8736


### Fechando o navegador

In [8]:
browser.quit()

## Atualizando a base com as novas cotações

In [9]:
# Importando a lista de produtos
df = pd.read_excel("Produtos.xlsx")
df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


### Realizando os calculos para atualizar o preço final

In [10]:
df.loc[df['Moeda'] == "Dólar", "Cotação"] = float(cotacaoDolar)
df.loc[df['Moeda'] == "Euro", "Cotação"] = float(cotacaoEuro)
df.loc[df['Moeda'] == "Ouro", "Cotação"] = float(cotacaoOuro)

# Atualizando preços (Valor original *  cotação)
df['Preço de Compra'] = df['Preço Original'] * df['Cotação']

# Preço final (Preço de compra * Margem)
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']

# transformando em R$
df['Preço de Venda'] = df['Preço de Venda'].map("R${:.2f}".format)

In [11]:
df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.873600,4873.551264,1.40,R$6822.97
1,Carro Renault,4500.00,Euro,5.261295,23675.826960,2.00,R$47351.65
2,Notebook Dell,899.99,Dólar,4.873600,4386.191264,1.70,R$7456.53
3,IPhone,799.00,Dólar,4.873600,3894.006400,1.70,R$6619.81
4,Carro Fiat,3000.00,Euro,5.261295,15783.884640,1.90,R$29989.38
5,Celular Xiaomi,480.48,Dólar,4.873600,2341.667328,2.00,R$4683.33
6,Joia 20g,20.00,Ouro,304.390000,6087.800000,1.15,R$7000.97


### Exportando nova base

In [12]:
df.to_excel("ProdutosNovo.xlsx", index=False)